In [1]:
# Go to directory: soccer-stars-django-webapp/server/backend
# Run in terminal: daphne -p 8000 backend.asgi:application

In [1]:
import nest_asyncio
nest_asyncio.apply()
import numpy as np
from typing import List
import asyncio
import json
from websocket import create_connection
from websocket import WebSocket

In [2]:
import os
os.chdir("/home/david/Documents/data_science/projects/soccer-stars-django-webapp/server/backend")

In [3]:
from match.match_logic import game_entities, formations, motion, utils

In [4]:
# Create a connection to the WebSocket
ws = create_connection("ws://localhost:8000/ws/match/")

In [5]:
def test_get_formation(ws: WebSocket):
    

    # Send the initial formation payload
    formation_payload = {
        "action": "create_formation",
        "left_formation": "formation1",
        "right_formation": "formation2"
    }
    ws.send(json.dumps(formation_payload))

    # Receive the response for initial formation
    result = ws.recv()

    # Parse the response to get the initial positions
    response_data = json.loads(result)

    return response_data

def test_arrow(ws: WebSocket, cap_idx, arrow_power, angle, initial_positions: List):
    """
        # Send the arrow submission payload
    arrow_payload = {
        "action": "submit_arrow",
        "cap_idx": 0,
        "arrow_power": 190,
        "angle": 15,
        "positions": initial_positions
    }
    ws.send(json.dumps(arrow_payload))

    # Receive the response for the arrow submission
    result = ws.recv()
    print("Received arrow submission: '%s'" % result)

    # Close the WebSocket connection
    ws.close()
    """
    
    # Create a connection to the WebSocket
    ws = create_connection("ws://localhost:8000/ws/match/")

    # Send the arrow submission payload
    arrow_payload = {
        "action": "submit_arrow",
        "cap_idx": cap_idx,
        "arrow_power": arrow_power,
        "angle": angle,
        "positions": initial_positions
    }
    ws.send(json.dumps(arrow_payload))

    # Receive the response for the arrow submission
    result = ws.recv()

    return json.loads(result)

In [6]:
# ---------------------------------- #
#   Start Game
# ---------------------------------- #
json_initial_positions = test_get_formation(ws=ws)
initial_positions = json_initial_positions.get("initial_positions")

In [7]:
json_initial_positions

{'initial_positions': [[66.20689392089844, 540.0],
  [297.9310302734375, 254.11764526367188],
  [297.9310302734375, 825.88232421875],
  [516.413818359375, 540.0],
  [728.27587890625, 540.0],
  [1787.586181640625, 381.1764831542969],
  [1787.586181640625, 698.8235473632812],
  [1522.758544921875, 540.0],
  [1191.72412109375, 190.58824157714844],
  [1191.72412109375, 889.4117431640625],
  [960.0, 540.0]]}

In [8]:
%%time
json_xhist = test_arrow(
    ws=ws, 
    cap_idx=0,
    arrow_power=190,
    angle=180,
    initial_positions=initial_positions
)

CPU times: user 65.5 ms, sys: 2.33 ms, total: 67.8 ms
Wall time: 161 ms


In [9]:
Xhist = json_xhist.get("positions")

In [17]:
json_xhist = test_arrow(
    ws=ws, 
    cap_idx=0,
    arrow_power=190,
    angle=210,
    initial_positions=Xhist[-1]
)
Xhist2 = json_xhist.get("positions")

# Visualize Game

In [10]:
w = 1920
h = 1080
margin = 300
boundaries = (w, h)
ratio_radii_cap_h = 1./17
radii = int(h * ratio_radii_cap_h)
goal_size = int(radii * 6)
goal_depth = int(radii * 2)
ur = utils.UtilsRender(window_size=boundaries, goal_depth=goal_depth, goal_size=goal_size)

In [11]:
n = len(initial_positions)
radii = 63
radii_ball = 31
R = np.array([radii for _ in range(n)])
R[-1] = radii_ball
ur.render_field_motion(positions=Xhist, radius=R, add_delay=10)